In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp -r /content/drive/MyDrive/isic/original .

In [3]:
!cp -r /content/drive/MyDrive/isic/src .

In [4]:
# !cat /proc/cpuinfo

In [5]:
!pip install -q torcheval timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 18.7 MB/s eta 0:00:00


In [6]:
import os
import sys
import random
import pathlib
import numpy as np
import pandas as pd
import torch
sys.path.append("./src")
from cross_validation import cross_validation
from image_train import set_seed, train_model, evaluate_model
from comp_score import comp_score

In [7]:
CV_MODE = "sgkf"
N_SPLITS = 10

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(DEVICE)

BATCH_SIZE = 16
SEED = 42
EPOCHS = 30
IMG_SIZE = 224

WEIGHT_DECAY = 1e-6
LR = 1e-5
NUM_WORKERS = 4

TIMM_PARAMS = {
    "model_name":  "convnext_base.fb_in22k_ft_in1k",#"eva02_small_patch14_336.mim_in22k_ft_in1k",
    "pretrained": True,
}

DATA_DIR = pathlib.Path("/content/original")
OUT_DIR = pathlib.Path("/content/drive/MyDrive/isic/outputs")

cuda


In [8]:
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False #True

In [9]:
df_train = pd.read_csv(DATA_DIR / "train-metadata.csv")
df_train = cross_validation(
    df_train=df_train,
    mode=CV_MODE,
    n_splits=N_SPLITS,
)
# df_train["target"] = ~df_train["lesion_id"].isna()

file_hdf = DATA_DIR / "train-image.hdf5"

# df_train = df_train.groupby("patient_id").sample(1, random_state=SEED, replace=True)

<ipython-input-9-ed7d235813d6>:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv(DATA_DIR / "train-metadata.csv")


In [10]:
list_df_preds = []
scores = []
for fold in range(0, N_SPLITS):
    df_train_fold = df_train[df_train["fold"] != fold].reset_index(drop=True)
    df_valid_fold = df_train[df_train["fold"] == fold].reset_index(drop=True)

    model = train_model(
        df_train=df_train_fold,
        df_valid=df_valid_fold,
        device=DEVICE,
        batch_size=BATCH_SIZE,
        timm_params=TIMM_PARAMS,
        epochs=EPOCHS,
        file_hdf=file_hdf,
        img_size=IMG_SIZE,
        num_workers=NUM_WORKERS,
        optimizer_type="AdamW",
        optimizer_params={"lr": LR, "weight_decay": WEIGHT_DECAY},
        #scheduler_type="none",
        #scheduler_params={},
        scheduler_type="CosineLRScheduler",
        scheduler_params={},
        early_stopping=False,
    )
    torch.save(model.state_dict(), OUT_DIR / f"image_{fold}.pt")

    preds = evaluate_model(
        model=model,
        df_valid=df_valid_fold,
        device=DEVICE,
        batch_size=BATCH_SIZE,
        file_hdf=file_hdf,
        img_size=IMG_SIZE,
        num_workers=NUM_WORKERS,
    )
    df_preds = df_valid_fold[["isic_id"]].copy()
    df_preds["pred"] = preds.numpy()
    list_df_preds.append(df_preds)

    score = comp_score(df_valid_fold[["target"]], pd.DataFrame(preds.numpy(), columns=["prediction"]), "")
    scores.append(score)
    print(f"fold: {fold} - Partial AUC Score: {score:.5f}")

df_preds = pd.concat(list_df_preds)
df_preds.to_parquet(OUT_DIR / "preds.parquet")

print(f"over all score: {np.mean(scores)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/354M [00:00<?, ?B/s]

100%|██████████| 39/39 [00:01<00:00, 20.50it/s, epoch=1, valid_Loss=0.471]


Validation AUROC Improved (-inf ---> 0.9705707491082045)


100%|██████████| 39/39 [00:01<00:00, 20.79it/s, epoch=2, valid_Loss=0.425]


...

Validation AUROC: 0.9170625


100%|██████████| 53/53 [00:02<00:00, 20.63it/s, epoch=30, valid_Loss=0.234]


Validation AUROC: 0.9170625


100%|██████████| 2630/2630 [02:02<00:00, 21.43it/s, Valid_Loss=0.226]


fold: 9 - Partial AUC Score: 0.13926
over all score: 0.15501692268702222


In [11]:
from google.colab import runtime
runtime.unassign()